In [1]:
import pandas as pd
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import tensorflow as tf
from keras.layers import Dense, Input, Activation
from keras.layers import BatchNormalization,Add,Dropout
from keras.optimizers import Adam
from keras.models import Model, load_model
from keras import callbacks
from keras import backend as K
from keras.layers.advanced_activations import LeakyReLU
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings(action="ignore",category=DeprecationWarning)
warnings.filterwarnings(action="ignore",category=FutureWarning)
import os
import gc
#%cd /kaggle/input/champs-scalar-coupling
#print(os.listdir("."))
# Any results you write to the current directory are saved as output.

Using TensorFlow backend.


In [2]:
df_train =pd.read_csv('../input/train.csv')
df_test  =pd.read_csv('../input/test.csv')
df_struct=pd.read_csv('../input/structures.csv')

df_train_sub_charge=pd.read_csv('../input/mulliken_charges.csv')
df_train_sub_tensor=pd.read_csv('../input/magnetic_shielding_tensors.csv')

In [3]:
''' 
Map atom info from the structures.csv into the train/test files
'''
import psutil
import os

def map_atom_info(df_1,df_2, atom_idx):
    print('Mapping...', df_1.shape, df_2.shape, atom_idx)
    df = pd.merge(df_1, df_2.drop_duplicates(subset=['molecule_name', 'atom_index']), how = 'left',
                  left_on  = ['molecule_name', f'atom_index_{atom_idx}'],
                  right_on = ['molecule_name',  'atom_index'])
    df = df.drop('atom_index', axis=1)
    return df

def show_ram_usage():
    py = psutil.Process(os.getpid())
    print('RAM usage: {} GB'.format(py.memory_info()[0]/2. ** 30))

show_ram_usage()

for atom_idx in [0,1]:
    df_train = map_atom_info(df_train,df_struct, atom_idx)
    df_train = map_atom_info(df_train,df_train_sub_charge, atom_idx)
    df_train = map_atom_info(df_train,df_train_sub_tensor, atom_idx)
    df_train = df_train.rename(columns={'atom': f'atom_{atom_idx}',
                                        'x': f'x_{atom_idx}',
                                        'y': f'y_{atom_idx}',
                                        'z': f'z_{atom_idx}',
                                        'mulliken_charge': f'charge_{atom_idx}',
                                        'XX': f'XX_{atom_idx}',
                                        'YX': f'YX_{atom_idx}',
                                        'ZX': f'ZX_{atom_idx}',
                                        'XY': f'XY_{atom_idx}',
                                        'YY': f'YY_{atom_idx}',
                                        'ZY': f'ZY_{atom_idx}',
                                        'XZ': f'XZ_{atom_idx}',
                                        'YZ': f'YZ_{atom_idx}',
                                        'ZZ': f'ZZ_{atom_idx}',})
    df_test = map_atom_info(df_test,df_struct, atom_idx)
    df_test = df_test.rename(columns={'atom': f'atom_{atom_idx}',
                                'x': f'x_{atom_idx}',
                                'y': f'y_{atom_idx}',
                                'z': f'z_{atom_idx}'})
    #add some features
    
    df_struct['c_x']=df_struct.groupby('molecule_name')['x'].transform('mean')
    df_struct['c_y']=df_struct.groupby('molecule_name')['y'].transform('mean')
    df_struct['c_z']=df_struct.groupby('molecule_name')['z'].transform('mean')
    df_struct['atom_n']=df_struct.groupby('molecule_name')['atom_index'].transform('max')
    
    show_ram_usage()
    print(df_train.shape, df_test.shape)

RAM usage: 0.9843254089355469 GB
Mapping... (4658147, 6) (2358657, 6) 0
Mapping... (4658147, 10) (1533537, 3) 0
Mapping... (4658147, 11) (1533537, 11) 0
Mapping... (2505542, 5) (2358657, 6) 0
RAM usage: 1.7024726867675781 GB
(4658147, 20) (2505542, 9)
Mapping... (4658147, 20) (2358657, 10) 1
Mapping... (4658147, 28) (1533537, 3) 1
Mapping... (4658147, 29) (1533537, 11) 1
Mapping... (2505542, 9) (2358657, 10) 1
RAM usage: 2.4983367919921875 GB
(4658147, 38) (2505542, 17)


In [5]:
case = '1JHC'

In [6]:
FE021_train_1JHC = pd.read_parquet('../data/FE021/FE021-train-1JHC.parquet', engine='pyarrow')
FE021_test_1JHC = pd.read_parquet('../data/FE021/FE021-test-1JHC.parquet', engine='pyarrow')

FE021_train_1JHN = pd.read_parquet('../data/FE021/FE021-train-1JHN.parquet', engine='pyarrow')
FE021_test_1JHN = pd.read_parquet('../data/FE021/FE021-test-1JHN.parquet', engine='pyarrow')

FE021_train_2JHH = pd.read_parquet('../data/FE021/FE021-train-2JHH.parquet', engine='pyarrow')
FE021_test_2JHH = pd.read_parquet('../data/FE021/FE021-test-2JHH.parquet', engine='pyarrow')

FE021_train_2JHC = pd.read_parquet('../data/FE021/FE021-train-2JHC.parquet', engine='pyarrow')
FE021_test_2JHC = pd.read_parquet('../data/FE021/FE021-test-2JHC.parquet', engine='pyarrow')

FE021_train_2JHN = pd.read_parquet('../data/FE021/FE021-train-2JHN.parquet', engine='pyarrow')
FE021_test_2JHN = pd.read_parquet('../data/FE021/FE021-test-2JHN.parquet', engine='pyarrow')

FE021_train_3JHH = pd.read_parquet('../data/FE021/FE021-train-3JHH.parquet', engine='pyarrow')
FE021_test_3JHH = pd.read_parquet('../data/FE021/FE021-test-3JHH.parquet', engine='pyarrow')

FE021_train_3JHC = pd.read_parquet('../data/FE021/FE021-train-3JHC.parquet', engine='pyarrow')
FE021_test_3JHC = pd.read_parquet('../data/FE021/FE021-test-3JHC.parquet', engine='pyarrow')

FE021_train_3JHN = pd.read_parquet('../data/FE021/FE021-train-3JHN.parquet', engine='pyarrow')
FE021_test_3JHN = pd.read_parquet('../data/FE021/FE021-test-3JHN.parquet', engine='pyarrow')


In [7]:
df_train.sort_values('id', inplace=True)
df_test.sort_values( 'id', inplace=True)
df_train.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,atom_0,x_0,y_0,z_0,...,x_farthest_0,y_farthest_0,z_farthest_0,min_distance_x,atom_index_farthest_1,distance_farthest_1,x_farthest_1,y_farthest_1,z_farthest_1,min_distance_y
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076,H,0.002150,-0.006031,0.001976,...,-0.523814,1.437933,0.906397,1.091953,1,1.091953,0.002150,-0.006031,0.001976,1.091946
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570,H,0.002150,-0.006031,0.001976,...,-0.523814,1.437933,0.906397,1.091953,3,1.783158,-0.540815,1.447527,-0.876644,1.091952
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548,H,0.002150,-0.006031,0.001976,...,-0.523814,1.437933,0.906397,1.091953,2,1.783158,1.011731,1.463751,0.000277,1.091946
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543,H,0.002150,-0.006031,0.001976,...,-0.523814,1.437933,0.906397,1.091953,1,1.783157,0.002150,-0.006031,0.001976,1.091948
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074,H,1.011731,1.463751,0.000277,...,-0.540815,1.447527,-0.876644,1.091952,1,1.091953,0.002150,-0.006031,0.001976,1.091946


In [8]:
df_train_case = df_train.loc[df_train.type==case, :].reset_index(drop=True)
df_test_case = df_test.loc[df_test.type==case, :].reset_index(drop=True)

In [9]:
len(df_train_case), len(FE021_train_1JHC), len(df_test_case)

(709416, 709416, 380609)

In [10]:
#Impute NA with mean
MEAN = pd.concat([FE021_train_1JHC,FE021_test_1JHC]).mean()
FE021_train_1JHC.fillna( value=MEAN, inplace=True )
FE021_test_1JHC.fillna( value=MEAN, inplace=True )
FE021_train_1JHC.head()

,10th_closest_to_1_valence_x_cube_inv_dist,12th_closest_to_1_valence,13th_closest_to_1_exact_mass,17th_closest_to_0_spin_multiplicity,17th_closest_to_0_valence,17th_closest_to_0_valence_x_cube_inv_dist,17th_closest_to_1_spin_multiplicity,18th_closest_to_0_exact_mass,19th_closest_to_0_spin_multiplicity,19th_closest_to_1_spin_multiplicity_x_cube_inv_dist,...,tor_ang_2leftleft_count,tor_ang_2leftleft_max,tor_ang_2leftleft_min,val_not_0_mean,val_not_1_mean,val_not_1_std,yukawa_H.x,yukawa_H.y,yukawa_N.x,yukawa_O.y
0,0.08638,1.969477,6.210087,0.001063,1.538449,0.013848,0.001074,3.800028,0.00024,0.000003,...,4.615952,125.460482,-124.658907,1.75,1.0,0.0,0.504328,1.342247,0.000000,0.0
1,0.08638,1.969477,6.210087,0.001063,1.538449,0.013848,0.001074,3.800028,0.00024,0.000003,...,4.615952,125.460482,-124.658907,1.75,1.0,0.0,0.504327,1.342247,0.000000,0.0
2,0.08638,1.969477,6.210087,0.001063,1.538449,0.013848,0.001074,3.800028,0.00024,0.000003,...,4.615952,125.460482,-124.658907,1.75,1.0,0.0,0.504323,1.342247,0.000000,0.0
3,0.08638,1.969477,6.210087,0.001063,1.538449,0.013848,0.001074,3.800028,0.00024,0.000003,...,4.615952,125.460482,-124.658907,1.75,1.0,0.0,0.504323,1.342247,0.000000,0.0
4,0.08638,1.969477,6.210087,0.001063,1.538449,0.013848,0.001074,3.800028,0.00024,0.000003,...,4.615952,125.460482,-124.658907,1.50,1.0,0.0,0.000000,0.344177,0.108789,0.0


In [11]:
# print(df_train.shape)
df_train_case_final = pd.concat( [df_train_case,FE021_train_1JHC], axis=1 )
df_test_case_final  = pd.concat( [df_test_case ,FE021_test_1JHC ], axis=1 )

In [12]:
len(df_train_case_final), len(df_test_case_final)

(709416, 380609)

In [13]:
extra_featus =  list(df_train_case_final.columns[81:])
extra_featus[:10]

['24th_closest_to_1_spin_multiplicity',
 '24th_closest_to_1_spin_multiplicity_x_cube_inv_dist',
 '25th_closest_to_0_spin_multiplicity_x_cube_inv_dist',
 '26th_closest_to_0_spin_multiplicity_x_cube_inv_dist',
 '2nd_closest_to_0_exact_mass',
 '2nd_closest_to_0_valence',
 '2nd_closest_to_0_valence_x_cube_inv_dist',
 '2nd_closest_to_1',
 '2nd_closest_to_1_exact_mass',
 '2nd_closest_to_1_valence']

## Neural Network definition

In [14]:
def create_nn_model(input_shape):
    inp = Input(shape=(input_shape,))
    x = Dense(1024)(inp)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.10)(x)
    x = Dropout(0.2)(x)
    x = Dense(1024)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.10)(x)
    x = Dropout(0.2)(x)
    x = Dense(1024)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.10)(x)
    x = Dropout(0.2)(x)
#     x = Dense(256)(x)
#     x = BatchNormalization()(x)
#     x = LeakyReLU(alpha=0.05)(x)
#     #x = Dropout(0.4)(x)
#     x = Dense(256)(x)
#     x = BatchNormalization()(x)
#     x = LeakyReLU(alpha=0.05)(x)
    #x = Dropout(0.4)(x)
    x = Dense(1024)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.10)(x)
    #x = Dropout(0.4)(x)
    out1 = Dense(2, activation="linear",name='outM2')(x)#mulliken charge 2
    out2 = Dense(6, activation="linear",name='outT6')(x)#tensor 6(xx,yy,zz)
    out3 = Dense(12, activation="linear",name='outT12')(x)#tensor 12(others) 
    x = Dense(256)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.10)(x)
    #x = Dropout(0.2)(x)
    x = Dense(128)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.10)(x)
#     x = Dense(128)(x)
#     x = BatchNormalization()(x)
#     x = LeakyReLU(alpha=0.05)(x)
    #x = Dropout(0.2)(x)
    out = Dense(1, activation="linear",name='out')(x)#scalar_coupling_constant    
    model = Model(inputs=inp, outputs=[out,out1,out2,out3])
    return model

In [15]:
nn_model=create_nn_model( 10 )
nn_model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 10)           0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 1024)         11264       input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 1024)         4096        dense_1[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyR

## Plot Function
I rely a lot on loss plots to detect when learning has stopped as well as when overfitting begins.

In [16]:
def plot_history(history, label):
    plt.plot(history.history['loss'][-100:])
    plt.plot(history.history['val_loss'][-100:])
    plt.title('Loss for %s' % label)
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    _= plt.legend(['Train','Validation'], loc='upper left')
    plt.show()

## Main Routine

In [ ]:
from datetime import datetime

#mol_types=df_train["type"].unique()
mol_types = ['1JHC']
cv_score=[]
cv_score_total=0 
epoch_n = 500 # 300
verbose = 2
batch_size = 1024 # 512
    
# Set to True if we want to train from scratch.  False will reuse saved models as a starting point.
retrain = False


# Set up GPU preferences
config = tf.ConfigProto( device_count = {'GPU': 2} ) 
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.6
sess = tf.Session(config=config) 
K.set_session(sess)

start_time=datetime.now()

# Loop through each molecule type
for mol_type in mol_types:
    model_name_rd = ('../keras-neural-net-for-champs/molecule_model_%s.hdf5' % mol_type)
    model_name_wrt = ('molecule_model_%s.hdf5' % mol_type)
    print('Training %s' % mol_type, 'out of', mol_types, '\n')
    
    df_train_ = df_train_case_final[ df_train_case_final["type"]==mol_type]
    df_test_  = df_test_case_final [ df_test_case_final["type"]==mol_type]
    splits = df_train_.id%5

    # Here's our best features.  We think.
    input_features=extra_featus

    # Standard Scaler from sklearn does seem to work better here than other Scalers
    input_data=StandardScaler().fit_transform(pd.concat([df_train_.loc[:,input_features],df_test_.loc[:,input_features]]))
    
    target_data=df_train_.loc[:,"scalar_coupling_constant"].values
    target_data_1=df_train_.loc[:,["charge_0","charge_1"]]
    target_data_2=df_train_.loc[:,["XX_0","YY_0","ZZ_0","XX_1","YY_1","ZZ_1"]]
    target_data_3=df_train_.loc[:,["YX_0","ZX_0","XY_0","ZY_0","XZ_0","YZ_0","YX_1","ZX_1","XY_1","ZY_1","XZ_1","YZ_1"]]
    
    #following parameters should be adjusted to control the loss function
    #if all parameters are zero, attractors do not work. (-> simple neural network)
    m1=2
    m2=4
    m3=1
    target_data_1=m1*(StandardScaler().fit_transform(target_data_1))
    target_data_2=m2*(StandardScaler().fit_transform(target_data_2))
    target_data_3=m3*(StandardScaler().fit_transform(target_data_3))
    
    # Simple split to provide us a validation set to do our CV checks with
    #train_index, cv_index = train_test_split(np.arange(len(df_train_)),random_state=111, test_size=0.05)
    train_index = np.where(splits != 0)[0]
    cv_index = np.where(splits == 0)[0]
    
    # Split all our input and targets by train and cv indexes
    train_input=input_data[train_index]
    cv_input=input_data[cv_index]
    train_target=target_data[train_index]
    cv_target=target_data[cv_index]
    train_target_1=target_data_1[train_index]
    cv_target_1=target_data_1[cv_index]
    train_target_2=target_data_2[train_index]
    cv_target_2=target_data_2[cv_index]
    train_target_3=target_data_3[train_index]
    cv_target_3=target_data_3[cv_index]
    test_input=input_data[len(df_train_):,:]

    # Build the Neural Net
    nn_model=create_nn_model(train_input.shape[1])
    
    # If retrain==False, then we load a previous saved model as a starting point.
    if retrain:
        nn_model = load_model(model_name_rd)
        
    nn_model.compile(loss='mae', optimizer=Adam())#, metrics=[auc])
    #nn_model.compile(loss='mean_squared_error', optimizer=Adam())#, metrics=[auc])
    
    # Callback for Early Stopping... May want to raise the min_delta for small numbers of epochs
    es = callbacks.EarlyStopping(monitor='val_out_loss', min_delta=0.0005, patience=30,verbose=1, mode='auto', restore_best_weights=True)
    # Callback for Reducing the Learning Rate... when the monitor levels out for 'patience' epochs, then the LR is reduced
    rlr = callbacks.ReduceLROnPlateau(monitor='val_out_loss', factor=0.3333,patience=15, min_lr=1e-6, mode='auto', verbose=1)
    # Save the best value of the model for future use
    sv_mod = callbacks.ModelCheckpoint(model_name_wrt, monitor='val_out_loss', save_best_only=True, period=1)

    print (train_input.shape, cv_input.shape)
    history = nn_model.fit(train_input,[train_target,train_target_1,train_target_2,train_target_3], 
                           validation_data=(cv_input,[cv_target,cv_target_1,cv_target_2,cv_target_3]), 
                           callbacks=[es, rlr, sv_mod],
                           epochs=epoch_n,
                           batch_size=batch_size,
                           verbose=verbose)
    
    cv_predict=nn_model.predict(cv_input)
    plot_history(history, mol_type)
    
    accuracy=np.mean(np.abs(cv_target-cv_predict[0][:,0]))
    print( accuracy,np.log(accuracy) )
    
    cv_score.append(np.log(accuracy))
    cv_score_total+=np.log(accuracy)
    
    # Predict on the test data set using our trained model
    test_predict=nn_model.predict(test_input)
    
    # for each molecule type we'll grab the predicted values
    test_prediction[df_test["type"]==mol_type]=test_predict[0][:,0]
    K.clear_session()

cv_score_total/=len(mol_types)

Training 1JHC out of ['1JHC'] 

(567515, 340) (141901, 340)
Instructions for updating:
Use tf.cast instead.
Train on 567515 samples, validate on 141901 samples
Epoch 1/500
 - 151s - loss: 60.7071 - out_loss: 58.1913 - outM2_loss: 0.2723 - outT6_loss: 1.5156 - outT12_loss: 0.7280 - val_loss: 40.7646 - val_out_loss: 35.7063 - val_outM2_loss: 1.2970 - val_outT6_loss: 2.9663 - val_outT12_loss: 0.7949
Epoch 2/500
 - 146s - loss: 4.4855 - out_loss: 2.1151 - outM2_loss: 0.2348 - outT6_loss: 1.4169 - outT12_loss: 0.7187 - val_loss: 3.8817 - val_out_loss: 1.6163 - val_outM2_loss: 0.2231 - val_outT6_loss: 1.3224 - val_outT12_loss: 0.7200
Epoch 3/500
 - 147s - loss: 3.7707 - out_loss: 1.5247 - outM2_loss: 0.1964 - outT6_loss: 1.3323 - outT12_loss: 0.7172 - val_loss: 3.3815 - val_out_loss: 1.2752 - val_outM2_loss: 0.1451 - val_outT6_loss: 1.2449 - val_outT12_loss: 0.7163
Epoch 4/500
 - 147s - loss: 3.5402 - out_loss: 1.3537 - outM2_loss: 0.1812 - outT6_loss: 1.2886 - outT12_loss: 0.7167 - val_loss

In [ ]:
test_input.shape

In [26]:
print ('Total training time: ', datetime.now() - start_time)

i=0
for mol_type in mol_types: 
    print(mol_type,": cv score is ",cv_score[i])
    i+=1
print("total cv score is",cv_score_total)

Total training time:  3:43:13.005300
1JHC : cv score is  -0.8547819
total cv score is -0.8547819256782532


## Prepare results for Submission

The total CV score matches Kaggle's score pretty closely.

In [ ]:
def submit(predictions):
    submit = pd.read_csv('../input/sample_submission.csv')
    print(len(submit), len(predictions))   
    submit["scalar_coupling_constant"] = predictions
    submit.to_csv("../submissions/keras-neural-net-1.csv", index=False)
submit(test_prediction)

print ('Total training time: ', datetime.now() - start_time)

i=0
for mol_type in mol_types: 
    print(mol_type,": cv score is ",cv_score[i])
    i+=1
print("total cv score is",cv_score_total)

In [ ]:
len(df_test_)

In [ ]:
mol_types=df_train["type"].unique()
cv_score=[]
cv_score_total=0
epoch_n = 50
verbose = 2
batch_size = 512
    
# Set to True if we want to train from scratch.  False will reuse saved models as a starting point.
retrain = True


# Set up GPU preferences
config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 2} ) 
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.6
sess = tf.Session(config=config) 
K.set_session(sess)

start_time=datetime.now()

# Loop through each molecule type
for mol_type in mol_types:
    model_name_rd = ('../keras-neural-net-for-champs/molecule_model_%s.hdf5' % mol_type)
    model_name_wrt = ('working/molecule_model_%s.hdf5' % mol_type)
    print('Training %s' % mol_type, 'out of', mol_types, '\n')
    
    df_train_ = df_train[df_train["type"]==mol_type]
    df_test_  = df_test [ df_test["type"]==mol_type]

    # Here's our best features.  We think.
    input_features=[
        "x_0","y_0","z_0","x_1","y_1","z_1","c_x","c_y","c_z",
                    'x_closest_0','y_closest_0','z_closest_0','x_closest_1','y_closest_1','z_closest_1',
                    "distance","distance_center0","distance_center1", "distance_c0","distance_c1","distance_f0","distance_f1",
                    "cos_c0_c1","cos_f0_f1","cos_center0_center1","cos_c0","cos_c1","cos_f0","cos_f1","cos_center0","cos_center1",
                    "atom_n",
                        'link0', 'dist_xyz', 'inv_dist0',
       'inv_dist1', 'inv_distP', 'inv_dist0R', 'inv_dist1R', 'inv_distPR',
       'inv_dist0E', 'linkM0', 'min_molecule_atom_0_dist_xyz',
       'mean_molecule_atom_0_dist_xyz', 'max_molecule_atom_0_dist_xyz',
       'sd_molecule_atom_0_dist_xyz', 'min_molecule_atom_1_dist_xyz',
       'mean_molecule_atom_1_dist_xyz', 'max_molecule_atom_1_dist_xyz',
       'distN0', 'distN1', 'bond_lengths_mean_y', 'bond_lengths_std_y',
       'yukawa_C.x', 'yukawa_H.x', 'yukawa_N.x', 'yukawa_O.x', 'yukawa_C.y',
       'yukawa_H.y', 'yukawa_N.y', 'yukawa_O.y', 'qm9_1', 'qm9_2', 'adH1',
       'adC1', 'adC2', 'adC3', 'adC4', 'adC5', 'adN1', 'adN2', 'F1dist1',
       'F1dist2', 'F1dist3', 'bond_atom', 'bond_distance', 'tertiary_angle_0',
       'tertiary_angle_1', 'tertiary_angle_2', 'tertiary_angle_3',
       'tertiary_angle_4', 'tertiary_angle_5', 'tertiary_angle_6',
       'tertiary_angle_7', 'tertiary_distance_0', 'tertiary_distance_1',
       'tertiary_distance_2', 'tertiary_distance_3', 'tertiary_distance_4',
       'tertiary_distance_5', 'tertiary_distance_6', 'tertiary_distance_7',
    ]

    # Standard Scaler from sklearn does seem to work better here than other Scalers
    input_data=StandardScaler().fit_transform(pd.concat([df_train_.loc[:,input_features],df_test_.loc[:,input_features]]))
    
    target_data=df_train_.loc[:,"scalar_coupling_constant"].values
    target_data_1=df_train_.loc[:,["charge_0","charge_1"]]
    target_data_2=df_train_.loc[:,["XX_0","YY_0","ZZ_0","XX_1","YY_1","ZZ_1"]]
    target_data_3=df_train_.loc[:,["YX_0","ZX_0","XY_0","ZY_0","XZ_0","YZ_0","YX_1","ZX_1","XY_1","ZY_1","XZ_1","YZ_1"]]
    
    #following parameters should be adjusted to control the loss function
    #if all parameters are zero, attractors do not work. (-> simple neural network)
    m1=2
    m2=4
    m3=1
    target_data_1=m1*(StandardScaler().fit_transform(target_data_1))
    target_data_2=m2*(StandardScaler().fit_transform(target_data_2))
    target_data_3=m3*(StandardScaler().fit_transform(target_data_3))
    
    # Simple split to provide us a validation set to do our CV checks with
    train_index, cv_index = train_test_split(np.arange(len(df_train_)),random_state=111, test_size=0.05)
    
    # Split all our input and targets by train and cv indexes
    train_input=input_data[train_index]
    cv_input=input_data[cv_index]
    train_target=target_data[train_index]
    cv_target=target_data[cv_index]
    train_target_1=target_data_1[train_index]
    cv_target_1=target_data_1[cv_index]
    train_target_2=target_data_2[train_index]
    cv_target_2=target_data_2[cv_index]
    train_target_3=target_data_3[train_index]
    cv_target_3=target_data_3[cv_index]
    test_input=input_data[len(df_train_):,:]

    # Build the Neural Net
    nn_model=create_nn_model(train_input.shape[1])
    
    # If retrain==False, then we load a previous saved model as a starting point.
    #if not retrain:
    nn_model = load_model(model_name_rd)
        
    nn_model.compile(loss='mae', optimizer=Adam(lr=0.00001))#, metrics=[auc])
    #nn_model.compile(loss='mean_squared_error', optimizer=Adam())#, metrics=[auc])
    
    # Callback for Early Stopping... May want to raise the min_delta for small numbers of epochs
    es = callbacks.EarlyStopping(monitor='val_out_loss', min_delta=0.0005, patience=30,verbose=1, mode='auto', restore_best_weights=True)
    # Callback for Reducing the Learning Rate... when the monitor levels out for 'patience' epochs, then the LR is reduced
    rlr = callbacks.ReduceLROnPlateau(monitor='val_out_loss', factor=0.3333,patience=15, min_lr=1e-6, mode='auto', verbose=1)
    # Save the best value of the model for future use
    sv_mod = callbacks.ModelCheckpoint(model_name_wrt, monitor='val_out_loss', save_best_only=True, period=1)

    history = nn_model.fit(train_input,[train_target,train_target_1,train_target_2,train_target_3], 
                           validation_data=(cv_input,[cv_target,cv_target_1,cv_target_2,cv_target_3]), 
                           callbacks=[es, rlr, sv_mod],
                           epochs=epoch_n,
                           batch_size=batch_size,
                           verbose=verbose)
    
    cv_predict=nn_model.predict(cv_input)
    plot_history(history, mol_type)
    
    accuracy=np.mean(np.abs(cv_target-cv_predict[0][:,0]))
    print( accuracy,np.log(accuracy) )
    
    cv_score.append(np.log(accuracy))
    cv_score_total+=np.log(accuracy)
    
    # Predict on the test data set using our trained model
    test_predict=nn_model.predict(test_input)
    
    # for each molecule type we'll grab the predicted values
    test_prediction[df_test["type"]==mol_type]=test_predict[0][:,0]
    K.clear_session()

cv_score_total/=len(mol_types)

In [ ]:
len(df_test), len(test_prediction[df_test["type"]==mol_type])

In [ ]:
# 1JHC : cv score is  -0.60937184
# 2JHH : cv score is  -2.0246258
# 1JHN : cv score is  -1.3377854
# 2JHN : cv score is  -2.0205796
# 2JHC : cv score is  -1.4633688
# 3JHH : cv score is  -1.8357394
# 3JHC : cv score is  -1.4595399
# 3JHN : cv score is  -2.2611656
# total cv score is -1.6265220269560814

In [ ]:
def submit(predictions):
    submit = pd.read_csv('../input/sample_submission.csv')
    print(len(submit), len(predictions))   
    submit["scalar_coupling_constant"] = predictions
    submit.to_csv("../submissions/keras-neural-net-1-ft.csv", index=False)
submit(test_prediction)

print ('Total training time: ', datetime.now() - start_time)

i=0
for mol_type in mol_types: 
    print(mol_type,": cv score is ",cv_score[i])
    i+=1
print("total cv score is",cv_score_total)